In [3]:
import pandas as pd
import numpy as np

In [75]:
# loading the data
data = pd.read_csv('datasets/nyu_data/selected/selected_5000_mse.csv')

In [16]:
# adding datasets/nyu_data/selected/ to all the cells of columns images and depths
data['images'] = 'datasets/nyu_data/selected/' + data['images']
data['depths'] = 'datasets/nyu_data/selected/' + data['depths']

In [6]:
df = data

In [18]:
import torch
from piq import brisque
from PIL import Image

# calculating the brisque score of an image given its path
def brisque_score(image_path):
    image = Image.open(image_path)
    image = np.array(image)
    image_tensor = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    # Move tensor to the appropriate device (CPU or GPU if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    image_tensor = image_tensor.to(device)
    # Calculate the BRISQUE score
    with torch.no_grad():  # Optionally disable gradient calculation
        brisque_score = brisque(image_tensor, data_range=1.0)
        return brisque_score.item()

In [19]:
! pip install piq

In [20]:
torch.cuda.is_available()

True

In [21]:
images_list = np.array(df['images'])

In [22]:
from tqdm import tqdm
brisque_list = []
for i in tqdm(range(len(images_list))):
    brisque_list.append(brisque_score(images_list[i]))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [20:11<00:00,  4.13it/s]


In [23]:
df['brisque'] = brisque_list

In [7]:
# getting the pearson correlation between the predicted and actual values
pearson_corr = np.corrcoef(df['mse'],df['brisque'])[0, 1]
print(f'Pearson Correlation: {pearson_corr}')

Pearson Correlation: -0.0358173037280832


In [74]:
data.to_csv('datasets/nyu_data/selected/selected_5000_mse_brisque.csv')

In [5]:
# loading the data
data = pd.read_csv('datasets/nyu_data/selected/selected_5000_mse_brisque.csv')

In [8]:
data[data['images'].str.contains("colours/")]['mse'].median()

0.170549132553393

In [9]:
data[data['images'].str.contains("colours_blurred/")]['mse'].median()

0.1658929301948107

In [10]:
data[data['images'].str.contains("colours_noisy/")]['mse'].median()

0.1712786520369834

In [13]:
data[data['images'].str.contains("colours_noisy_gaussian/")]['mse'].median()

0.1793792933692727


In [70]:
data[data['images'].str.contains("colours/")]['brisque'].mean()

35.07163653987469

In [71]:
data[data['images'].str.contains("colours_blurred/")]['brisque'].mean()

59.70570058766423

In [72]:
data[data['images'].str.contains("colours_noisy/")]['brisque'].mean()

88.83564661450954

In [73]:
data[data['images'].str.contains("colours_noisy_gaussian/")]['brisque'].mean()

71.93290867539703

In [78]:
# min max normalizing the mse values
data['mse'] = (data['mse'] - data['mse'].min()) / (data['mse'].max() - data['mse'].min())